In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch

from othello_utils import plot_single_board, to_string, to_int, int_to_label, string_to_label, OthelloBoardState

In [3]:
data = [
    [ 20, 19, 18, 10, 2, 1, 27, 3, 11, 12, 5, 21, 14, 33, 26, 4, 40, 28, 13, 7, 42, 47, 34, 39, 49, 50, 51, 9, 38, 48, 55, 46, 15, 23, 32, 45, 17, 22, 37, 31, 8, 25, 6, 58, 54, 52, 16, 41, 56, 35, 60, 29, 44, 57, 36, 43, 30, 24, 59, 53 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 11, 12, 34, 26, 9, 43, 25, 39, 4, 33, 35, 5, 21, 31, 38, 32, 47, 46, 37, 40, 52, 22, 41, 17, 54, 36, 29, 50, 42, 55, 28, 45, 48, 57, 44, 24, 15, 6, 30, 60, 49, 53, 14, 56, 13, 51, 58, 23, 8, 16, 7, 59 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 11, 12, 41, 40, 5, 28, 21, 50, 48, 39, 23, 4, 13, 42, 33, 34, 59, 32, 51, 35, 25, 9, 36, 60, 49, 58, 26, 56, 17, 46, 38, 45, 43, 30, 24, 16, 53, 44, 37, 22, 55, 31, 57, 47, 29, 6, 15, 54, 14, 8, 7, 52 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 11, 12, 41, 42, 4, 17, 43, 28, 29, 49, 9, 34, 40, 50, 13, 26, 57, 21, 25, 22, 35, 33, 15, 31, 39, 30, 5, 56, 14, 52, 24, 16, 44, 32, 8, 36, 60, 48, 55, 23, 7, 6, 47, 38, 46, 45, 37, 54, 53, 51, 59, 58 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 11, 12, 42, 34, 5, 4, 22, 32, 28, 26, 31, 40, 47, 21, 38, 43, 33, 46, 13, 49, 52, 54, 45, 39, 37, 51, 9, 23, 15, 7, 50, 25, 35, 29, 55, 41, 14, 36, 53, 44, 30, 16, 58, 24, 8, 59, 17, 57, 6, 48, 60, 56 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 11, 21, 9, 17, 22, 41, 12, 5, 49, 33, 25, 23, 13, 57, 6, 31, 50, 51, 24, 14, 39, 7, 4, 46, 40, 26, 15, 8, 53, 38, 34, 54, 32, 29, 45, 42, 58, 59, 55, 35, 28, 37, 48, 47, 36, 44, 30, 16, 60, 43, 52, 56 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 11, 21, 9, 33, 22, 23, 24, 14, 39, 38, 34, 28, 6, 17, 45, 47, 56, 42, 26, 41, 12, 25, 48, 7, 51, 30, 32, 57, 58, 15, 37, 4, 40, 31, 8, 29, 5, 55, 49, 53, 35, 36, 46, 50, 54, 13, 43, 52, 60, 16, 44, 59 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 11, 21, 22, 12, 4, 5, 34, 13, 6, 42, 41, 26, 33, 40, 50, 35, 28, 23, 48, 49, 15, 29, 32, 55, 36, 14, 16, 30, 43, 37, 39, 24, 17, 47, 56, 25, 31, 59, 57, 8, 54, 38, 7, 51, 46, 53, 45, 58, 60, 52, 44, 9 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 11, 21, 22, 33, 41, 34, 4, 32, 42, 23, 12, 5, 37, 9, 17, 25, 28, 39, 26, 40, 15, 14, 49, 16, 6, 31, 38, 47, 54, 57, 30, 48, 45, 51, 50, 55, 56, 29, 60, 46, 13, 53, 8, 59, 24, 43, 58, 7, 35, 52, 44, 36 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 11, 21, 22, 34, 13, 5, 35, 23, 6, 7, 24, 41, 4, 43, 50, 49, 48, 57, 59, 39, 9, 12, 8, 28, 47, 33, 40, 58, 56, 42, 36, 17, 38, 16, 29, 30, 44, 46, 32, 55, 25, 15, 14, 45, 53, 54, 26, 31, 37, 60, 51, 52 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 11, 21, 28, 12, 13, 23, 5, 26, 32, 14, 42, 38, 7, 39, 6, 15, 29, 51, 46, 35, 31, 4, 43, 25, 17, 44, 41, 48, 22, 36, 34, 9, 55, 53, 52, 49, 33, 50, 30, 8, 60, 47, 59, 58, 16, 24, 57, 56, 45, 37, 40, 54 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 11, 21, 34, 26, 9, 42, 13, 28, 29, 12, 43, 44, 50, 23, 32, 51, 4, 5, 52, 38, 22, 30, 6, 58, 49, 41, 33, 40, 37, 15, 35, 17, 56, 39, 16, 36, 7, 8, 47, 60, 14, 59, 45, 55, 25, 24, 48, 57, 54, 31, 46, 53 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 11, 21, 34, 33, 26, 35, 4, 9, 12, 25, 42, 41, 38, 49, 17, 39, 47, 32, 51, 5, 40, 60, 58, 45, 29, 43, 28, 13, 22, 50, 14, 36, 6, 23, 16, 46, 7, 48, 54, 15, 24, 55, 59, 8, 31, 37, 57, 53, 56, 52, 44, 30 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 11, 21, 42, 33, 13, 12, 38, 28, 32, 41, 50, 40, 49, 14, 34, 58, 5, 48, 7, 45, 47, 9, 57, 22, 35, 54, 43, 6, 46, 8, 26, 4, 23, 31, 39, 30, 16, 59, 60, 56, 36, 24, 15, 53, 55, 29, 37, 52, 17, 25, 44, 51 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 11, 21, 42, 51, 28, 34, 35, 40, 4, 26, 12, 23, 41, 44, 31, 14, 39, 48, 25, 22, 24, 46, 60, 32, 30, 50, 5, 16, 57, 37, 53, 15, 33, 47, 54, 13, 49, 17, 7, 38, 8, 36, 52, 43, 45, 6, 9, 59, 58, 56, 29, 55 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 11, 26, 9, 17, 34, 42, 41, 21, 13, 22, 25, 6, 50, 39, 28, 43, 44, 48, 23, 52, 60, 31, 49, 51, 59, 33, 35, 15, 5, 40, 38, 30, 8, 29, 7, 32, 58, 46, 24, 14, 36, 4, 55, 47, 12, 56, 54, 16, 37, 53, 45, 57 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 11, 26, 9, 33, 12, 34, 28, 4, 42, 22, 40, 35, 38, 17, 36, 39, 21, 51, 23, 15, 14, 49, 8, 37, 45, 13, 50, 16, 46, 41, 56, 48, 31, 32, 6, 29, 30, 54, 53, 24, 7, 47, 52, 60, 43, 44, 55, 5, 59, 58, 25, 57 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 11, 26, 9, 34, 13, 12, 32, 14, 6, 40, 28, 22, 4, 21, 23, 5, 29, 30, 41, 38, 48, 47, 42, 33, 15, 36, 17, 56, 25, 50, 24, 31, 37, 49, 44, 43, 52, 8, 55, 16, 58, 7, 46, 54, 57, 59, 45, 53, 39, 51, 60, 35 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 11, 26, 25, 34, 4, 39, 28, 12, 5, 9, 46, 21, 14, 13, 40, 29, 23, 41, 35, 6, 48, 22, 15, 16, 8, 47, 17, 43, 30, 38, 7, 57, 37, 31, 52, 24, 50, 45, 56, 33, 58, 42, 54, 36, 32, 53, 44, 59, 49, 55, 51, 60 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 11, 26, 31, 33, 12, 22, 34, 13, 15, 21, 28, 35, 29, 23, 30, 7, 24, 17, 38, 32, 44, 16, 14, 43, 5, 45, 9, 4, 25, 6, 8, 37, 46, 55, 51, 59, 54, 53, 40, 48, 47, 49, 52, 41, 42, 56, 36, 39, 58, 57, 50, 60 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 11, 26, 34, 33, 25, 35, 43, 52, 39, 31, 40, 41, 37, 48, 17, 46, 4, 9, 12, 45, 47, 21, 56, 38, 49, 55, 28, 5, 50, 32, 13, 23, 53, 6, 54, 42, 51, 14, 22, 15, 16, 30, 29, 44, 36, 59, 60, 57, 24, 8, 58, 7 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 11, 26, 41, 28, 25, 42, 29, 32, 31, 40, 49, 12, 17, 58, 47, 33, 43, 54, 21, 50, 57, 34, 39, 48, 55, 36, 44, 52, 35, 14, 59, 22, 23, 38, 5, 16, 56, 60, 15, 30, 24, 51, 4, 13, 6, 9, 37, 8, 7, 46, 53, 45 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 11, 26, 42, 51, 34, 33, 9, 12, 60, 32, 38, 17, 21, 22, 25, 39, 5, 13, 23, 24, 48, 35, 30, 45, 14, 47, 6, 36, 28, 43, 40, 56, 29, 31, 54, 55, 4, 53, 41, 15, 37, 44, 7, 16, 57, 52, 46, 8, 49, 50, 58, 59 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 11, 33, 4, 28, 32, 9, 25, 17, 34, 37, 29, 42, 31, 35, 40, 24, 30, 39, 44, 36, 16, 21, 41, 5, 43, 22, 45, 53, 47, 12, 14, 48, 55, 26, 46, 51, 38, 57, 49, 54, 58, 13, 56, 52, 60, 50, 15, 6, 23, 7, 8, 59 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 11, 33, 39, 26, 9, 47, 46, 45, 55, 21, 28, 29, 53, 17, 34, 40, 23, 43, 22, 14, 35, 54, 25, 38, 6, 31, 12, 56, 37, 36, 52, 51, 50, 13, 15, 41, 4, 59, 32, 5, 49, 57, 42, 48, 44, 24, 58, 16, 8, 7, 60, 30 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 11, 33, 39, 34, 38, 32, 42, 26, 25, 46, 48, 40, 9, 31, 41, 55, 50, 43, 28, 17, 37, 57, 53, 21, 56, 45, 58, 54, 36, 59, 12, 35, 14, 52, 49, 47, 29, 30, 22, 7, 13, 5, 6, 23, 44, 4, 8, 51, 15, 16, 24, 60 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 11, 33, 40, 17, 28, 21, 9, 49, 47, 35, 22, 54, 41, 14, 23, 16, 7, 42, 39, 38, 50, 51, 34, 29, 46, 12, 60, 26, 45, 15, 44, 6, 55, 30, 58, 8, 13, 5, 24, 4, 36, 43, 32, 52, 53, 57, 48, 25, 56, 37, 31, 59 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 11, 33, 41, 17, 38, 39, 40, 21, 12, 28, 14, 46, 26, 49, 35, 44, 31, 25, 5, 13, 9, 4, 42, 32, 37, 34, 57, 45, 53, 54, 29, 23, 55, 48, 30, 22, 47, 51, 6, 7, 43, 50, 59, 60, 15, 58, 56, 24, 8, 36, 52, 16 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 11, 33, 41, 17, 39, 50, 49, 46, 47, 21, 12, 22, 26, 42, 28, 48, 43, 36, 44, 31, 23, 40, 59, 32, 54, 51, 38, 5, 30, 45, 37, 15, 55, 34, 7, 53, 57, 16, 14, 58, 4, 8, 52, 24, 29, 6, 9, 13, 25, 35, 56, 60 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 11, 33, 41, 42, 39, 17, 43, 34, 29, 32, 9, 49, 28, 47, 38, 22, 40, 23, 37, 44, 50, 26, 12, 58, 51, 45, 35, 59, 21, 4, 16, 15, 46, 53, 54, 55, 57, 25, 14, 13, 48, 6, 60, 7, 52, 56, 5, 31, 8, 36, 24, 30 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 11, 33, 42, 12, 4, 26, 13, 28, 40, 34, 39, 48, 41, 49, 25, 51, 60, 32, 31, 43, 58, 38, 23, 21, 35, 57, 50, 6, 55, 29, 22, 5, 14, 52, 44, 56, 37, 46, 45, 30, 15, 54, 59, 16, 7, 36, 47, 8, 9, 24, 53, 17 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 11, 33, 42, 13, 41, 17, 6, 50, 58, 21, 9, 32, 37, 31, 4, 26, 25, 49, 34, 29, 43, 40, 12, 52, 44, 5, 57, 51, 22, 36, 38, 56, 39, 59, 28, 47, 35, 7, 48, 55, 14, 30, 60, 45, 54, 15, 24, 23, 8, 16, 46, 53 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 11, 34, 12, 25, 9, 33, 28, 23, 42, 40, 4, 21, 41, 5, 39, 46, 22, 29, 13, 14, 35, 50, 30, 47, 16, 36, 58, 51, 15, 59, 48, 57, 55, 8, 44, 54, 49, 6, 60, 17, 32, 37, 7, 56, 31, 26, 45, 38, 24, 43, 53, 52 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 11, 34, 12, 33, 42, 21, 4, 9, 40, 13, 32, 39, 6, 51, 25, 41, 48, 14, 29, 28, 35, 44, 60, 49, 36, 46, 15, 30, 24, 22, 47, 16, 23, 43, 45, 56, 50, 53, 7, 57, 5, 54, 26, 58, 38, 8, 55, 17, 37, 31, 59, 52 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 11, 34, 13, 33, 43, 22, 26, 21, 4, 25, 9, 52, 39, 12, 41, 32, 23, 17, 38, 24, 15, 47, 37, 14, 6, 16, 8, 48, 55, 28, 40, 54, 29, 45, 5, 57, 31, 35, 42, 56, 30, 51, 46, 36, 50, 49, 53, 7, 59, 58, 44, 60 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 11, 34, 35, 41, 4, 33, 32, 43, 51, 26, 22, 5, 49, 40, 13, 12, 29, 38, 31, 28, 25, 30, 24, 21, 48, 56, 14, 50, 55, 37, 46, 42, 44, 39, 36, 15, 6, 54, 45, 7, 9, 60, 53, 17, 58, 16, 57, 59, 8, 47, 52, 23 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 11, 34, 35, 41, 21, 29, 4, 36, 50, 48, 43, 57, 52, 5, 55, 49, 51, 26, 24, 33, 9, 25, 17, 28, 38, 12, 22, 40, 39, 47, 54, 13, 46, 45, 58, 32, 15, 59, 42, 8, 23, 56, 44, 30, 60, 16, 53, 14, 6, 7, 31, 37 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 11, 34, 42, 39, 41, 17, 28, 43, 29, 51, 40, 22, 12, 13, 9, 24, 52, 44, 30, 60, 4, 33, 50, 35, 46, 57, 38, 49, 16, 23, 25, 21, 32, 47, 55, 31, 36, 26, 37, 15, 48, 45, 53, 5, 8, 54, 56, 14, 58, 7, 59, 6 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 17, 9, 26, 11, 21, 25, 33, 40, 48, 49, 41, 13, 28, 29, 42, 50, 22, 47, 43, 52, 4, 5, 51, 39, 55, 46, 12, 59, 24, 57, 35, 30, 58, 15, 6, 16, 56, 8, 38, 37, 14, 32, 31, 23, 44, 36, 45, 53, 54, 60, 7 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 17, 9, 26, 25, 42, 11, 40, 41, 28, 29, 21, 43, 31, 39, 46, 13, 32, 38, 6, 5, 51, 33, 48, 37, 45, 54, 50, 59, 4, 57, 12, 47, 56, 49, 35, 55, 30, 22, 15, 14, 53, 24, 23, 16, 44, 52, 60, 36, 58, 8, 7 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 17, 9, 33, 11, 43, 35, 25, 26, 29, 38, 40, 31, 42, 39, 41, 44, 50, 13, 36, 21, 47, 30, 14, 32, 12, 23, 24, 7, 28, 51, 22, 59, 5, 6, 45, 46, 49, 16, 60, 58, 53, 55, 37, 4, 57, 48, 56, 54, 15, 8, 52 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 17, 9, 42, 11, 43, 21, 26, 22, 23, 12, 33, 39, 41, 32, 46, 49, 25, 28, 38, 47, 35, 15, 8, 44, 4, 14, 40, 30, 29, 37, 7, 55, 54, 36, 51, 50, 56, 31, 58, 48, 52, 53, 24, 5, 45, 59, 57, 60, 6, 13, 16 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 17, 11, 4, 9, 33, 12, 13, 38, 25, 26, 28, 21, 5, 6, 42, 35, 45, 32, 30, 50, 29, 24, 23, 22, 51, 40, 48, 56, 7, 43, 52, 31, 15, 8, 58, 59, 14, 16, 41, 47, 60, 49, 39, 46, 54, 53, 57, 36, 55, 37, 44 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 17, 11, 4, 9, 43, 13, 33, 42, 26, 44, 52, 12, 41, 38, 5, 60, 39, 40, 14, 15, 23, 28, 51, 50, 16, 21, 48, 25, 49, 58, 37, 55, 32, 45, 31, 46, 56, 57, 47, 7, 59, 30, 53, 54, 22, 29, 35, 24, 36, 8, 6 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 17, 11, 33, 32, 21, 9, 41, 48, 49, 14, 29, 50, 57, 42, 43, 59, 22, 13, 25, 44, 36, 31, 40, 39, 55, 28, 23, 15, 4, 24, 47, 12, 52, 35, 16, 38, 45, 26, 37, 58, 51, 7, 6, 46, 8, 56, 60, 5, 54, 53, 30 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 17, 11, 33, 32, 43, 35, 29, 9, 42, 36, 25, 28, 26, 31, 30, 24, 39, 41, 49, 56, 37, 38, 40, 48, 50, 22, 55, 44, 52, 13, 6, 60, 46, 4, 23, 47, 58, 45, 12, 5, 14, 59, 51, 54, 57, 16, 21, 7, 8, 15, 53 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 17, 11, 41, 26, 21, 14, 43, 12, 13, 5, 25, 33, 4, 50, 39, 9, 28, 52, 42, 51, 49, 48, 22, 35, 32, 46, 60, 23, 7, 40, 59, 57, 29, 38, 55, 6, 24, 8, 53, 31, 44, 16, 37, 30, 36, 45, 47, 56, 15, 54, 58 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 17, 11, 43, 12, 39, 40, 41, 9, 13, 33, 26, 25, 31, 42, 51, 50, 4, 6, 57, 22, 15, 59, 14, 21, 49, 24, 16, 23, 28, 58, 5, 44, 52, 60, 29, 56, 30, 48, 55, 46, 37, 7, 32, 8, 47, 45, 53, 36, 54, 35, 38 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 21, 11, 12, 4, 43, 22, 13, 52, 5, 6, 33, 38, 41, 26, 9, 17, 15, 23, 7, 39, 35, 8, 29, 36, 47, 32, 37, 24, 42, 31, 30, 40, 49, 46, 28, 14, 45, 50, 48, 56, 16, 54, 25, 44, 53, 58, 51, 60, 57, 59, 55 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 21, 11, 12, 14, 33, 32, 26, 4, 22, 31, 42, 28, 29, 35, 40, 23, 13, 25, 38, 17, 30, 48, 41, 47, 6, 43, 45, 9, 37, 5, 36, 39, 44, 15, 54, 55, 56, 51, 50, 7, 24, 46, 53, 57, 16, 59, 8, 49, 52, 58, 60 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 21, 11, 26, 14, 4, 9, 43, 25, 39, 28, 22, 29, 35, 13, 40, 48, 57, 12, 24, 56, 41, 50, 15, 23, 32, 44, 42, 30, 16, 17, 58, 7, 36, 49, 33, 51, 52, 46, 31, 59, 55, 38, 5, 54, 60, 47, 8, 6, 45, 53, 37 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 21, 11, 33, 32, 12, 13, 40, 4, 14, 22, 42, 49, 47, 6, 31, 48, 35, 51, 25, 43, 59, 46, 15, 44, 56, 8, 41, 36, 52, 58, 28, 26, 38, 9, 50, 45, 30, 54, 39, 60, 17, 29, 16, 5, 37, 57, 55, 24, 7, 53, 23 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 21, 11, 42, 50, 33, 13, 9, 17, 35, 41, 59, 32, 14, 36, 4, 7, 22, 12, 15, 48, 38, 6, 28, 49, 44, 51, 60, 8, 26, 25, 5, 29, 58, 43, 16, 39, 46, 57, 40, 53, 23, 30, 47, 55, 54, 37, 45, 52, 24, 31, 56 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 21, 11, 43, 4, 40, 35, 36, 28, 9, 52, 22, 42, 41, 48, 33, 17, 23, 12, 13, 16, 50, 51, 5, 14, 56, 59, 32, 55, 7, 30, 57, 31, 29, 49, 39, 6, 60, 38, 24, 44, 58, 8, 54, 47, 26, 25, 37, 45, 15, 46, 53 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 21, 12, 33, 28, 41, 26, 4, 40, 48, 5, 9, 17, 43, 35, 11, 52, 6, 49, 25, 55, 38, 45, 58, 50, 29, 31, 51, 30, 37, 42, 22, 47, 24, 44, 13, 14, 53, 23, 15, 7, 46, 39, 32, 59, 8, 57, 60, 16, 54, 36, 56 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 21, 13, 33, 26, 35, 41, 48, 36, 22, 11, 32, 17, 43, 50, 40, 39, 4, 28, 44, 38, 59, 57, 46, 52, 42, 5, 51, 15, 56, 55, 58, 47, 8, 29, 54, 12, 6, 37, 14, 31, 23, 60, 49, 16, 24, 9, 7, 53, 25, 30, 45 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 21, 14, 17, 11, 43, 12, 33, 42, 7, 13, 28, 52, 22, 23, 50, 35, 6, 58, 44, 29, 16, 26, 51, 15, 60, 9, 49, 32, 40, 30, 4, 41, 31, 48, 5, 38, 24, 25, 37, 36, 39, 56, 47, 46, 45, 57, 55, 59, 54, 53, 8 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 21, 14, 22, 12, 39, 23, 43, 33, 24, 28, 4, 42, 32, 11, 26, 16, 8, 25, 41, 51, 15, 50, 6, 44, 52, 46, 29, 9, 47, 37, 38, 60, 13, 36, 45, 35, 49, 7, 40, 57, 48, 55, 58, 30, 31, 56, 53, 59, 54, 5, 17 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 21, 14, 33, 12, 28, 22, 26, 32, 42, 40, 35, 11, 41, 50, 6, 48, 37, 5, 58, 9, 49, 7, 17, 13, 47, 25, 31, 43, 39, 57, 4, 46, 53, 51, 60, 59, 44, 23, 56, 38, 15, 52, 30, 55, 8, 16, 45, 36, 29, 24, 54 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 21, 14, 43, 13, 22, 35, 32, 28, 44, 29, 5, 26, 17, 4, 31, 37, 33, 51, 7, 41, 42, 39, 60, 12, 24, 40, 47, 9, 23, 11, 30, 48, 49, 38, 46, 45, 53, 56, 57, 54, 55, 36, 15, 50, 59, 16, 52, 6, 8, 58, 25 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 26, 9, 28, 11, 12, 13, 21, 32, 33, 29, 14, 5, 17, 7, 15, 38, 42, 4, 45, 50, 35, 6, 31, 46, 23, 24, 49, 39, 8, 30, 41, 22, 16, 40, 25, 56, 57, 43, 36, 44, 52, 58, 48, 47, 37, 51, 60, 59, 53, 54, 55 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 26, 9, 33, 25, 42, 32, 40, 49, 21, 50, 38, 39, 47, 13, 35, 22, 17, 28, 29, 41, 23, 30, 37, 48, 6, 15, 59, 5, 36, 55, 43, 44, 54, 11, 24, 7, 8, 45, 4, 14, 31, 16, 53, 46, 58, 56, 57, 12, 52, 51, 60 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 26, 11, 21, 9, 43, 22, 12, 4, 42, 31, 28, 52, 33, 39, 41, 49, 57, 13, 14, 6, 46, 32, 7, 48, 38, 53, 44, 51, 5, 56, 35, 25, 37, 40, 60, 58, 54, 30, 50, 45, 29, 24, 23, 16, 47, 55, 15, 36, 59, 8, 17 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 26, 11, 42, 17, 43, 51, 25, 44, 39, 35, 33, 12, 41, 9, 29, 47, 59, 31, 49, 52, 32, 48, 22, 60, 21, 23, 14, 4, 37, 56, 24, 58, 57, 38, 36, 40, 13, 15, 45, 28, 6, 5, 55, 8, 16, 30, 50, 46, 54, 7, 53 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 26, 11, 43, 17, 32, 31, 33, 39, 47, 28, 9, 42, 21, 46, 35, 29, 25, 44, 36, 40, 22, 48, 56, 14, 49, 30, 38, 12, 54, 50, 4, 55, 13, 37, 15, 6, 53, 5, 51, 41, 23, 52, 59, 45, 57, 60, 7, 8, 16, 58, 24 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 26, 25, 31, 11, 12, 37, 43, 13, 40, 35, 28, 4, 29, 5, 42, 48, 47, 9, 6, 22, 14, 36, 21, 54, 41, 23, 32, 49, 33, 44, 15, 39, 24, 16, 58, 17, 55, 56, 51, 60, 7, 30, 59, 52, 45, 38, 57, 50, 46, 53, 8 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 26, 25, 32, 17, 28, 38, 11, 13, 42, 4, 5, 9, 39, 51, 12, 21, 33, 23, 22, 35, 41, 6, 7, 40, 37, 31, 50, 45, 47, 59, 49, 46, 24, 14, 29, 30, 36, 57, 56, 48, 60, 52, 15, 16, 54, 53, 58, 44, 8, 55, 43 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 26, 25, 42, 9, 32, 11, 28, 35, 41, 38, 36, 51, 46, 31, 4, 39, 60, 29, 40, 17, 37, 53, 33, 43, 23, 24, 21, 13, 12, 52, 6, 5, 44, 48, 22, 14, 55, 47, 54, 45, 15, 56, 49, 30, 7, 59, 57, 58, 16, 8, 50 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 33, 26, 17, 39, 47, 25, 41, 12, 42, 40, 49, 32, 35, 51, 11, 21, 22, 50, 4, 38, 52, 14, 31, 13, 6, 60, 43, 28, 29, 57, 59, 5, 58, 46, 53, 54, 56, 7, 15, 24, 48, 55, 16, 45, 23, 30, 37, 44, 8, 9, 36 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 33, 26, 35, 11, 12, 13, 28, 9, 32, 37, 17, 41, 5, 36, 49, 39, 14, 48, 46, 55, 47, 22, 43, 25, 50, 29, 31, 40, 45, 53, 57, 58, 15, 51, 52, 6, 24, 38, 23, 59, 54, 42, 30, 44, 56, 4, 21, 8, 60, 16, 7 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 33, 26, 35, 38, 12, 39, 31, 21, 47, 37, 40, 11, 46, 41, 32, 53, 54, 25, 49, 29, 13, 9, 23, 48, 43, 42, 51, 57, 58, 50, 56, 55, 45, 36, 30, 59, 52, 14, 4, 24, 17, 28, 5, 16, 15, 44, 60, 22, 8, 6, 7 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 33, 32, 21, 11, 17, 12, 4, 14, 39, 26, 25, 37, 40, 13, 7, 5, 31, 46, 47, 54, 41, 38, 43, 23, 45, 42, 6, 44, 28, 48, 50, 29, 49, 56, 55, 35, 22, 51, 52, 15, 36, 53, 57, 9, 30, 16, 24, 58, 59, 60, 8 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 33, 32, 26, 11, 42, 41, 50, 9, 40, 48, 28, 35, 36, 29, 17, 43, 22, 39, 46, 30, 57, 13, 47, 54, 49, 51, 59, 31, 4, 23, 52, 44, 15, 24, 55, 6, 25, 60, 7, 8, 21, 56, 53, 58, 37, 16, 5, 14, 12, 38, 45 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 33, 32, 26, 25, 42, 12, 31, 37, 4, 5, 35, 13, 17, 39, 45, 36, 22, 51, 11, 15, 29, 24, 43, 9, 38, 46, 41, 6, 23, 53, 7, 50, 49, 28, 58, 30, 47, 14, 16, 56, 21, 48, 44, 40, 52, 59, 60, 8, 57, 54, 55 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 33, 32, 39, 38, 22, 40, 21, 11, 45, 26, 43, 23, 15, 28, 17, 42, 47, 44, 48, 57, 16, 8, 35, 46, 53, 41, 37, 36, 49, 55, 31, 13, 7, 12, 14, 9, 51, 5, 54, 56, 25, 29, 52, 30, 50, 59, 24, 4, 60, 6, 58 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 33, 32, 42, 11, 31, 40, 12, 41, 21, 5, 17, 26, 39, 47, 55, 48, 56, 13, 29, 43, 6, 49, 35, 9, 57, 38, 14, 28, 37, 54, 51, 44, 50, 58, 4, 46, 52, 60, 25, 36, 30, 24, 23, 22, 59, 45, 53, 15, 7, 8, 16 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 33, 38, 9, 26, 41, 40, 42, 32, 25, 17, 48, 56, 47, 50, 39, 55, 35, 43, 13, 28, 57, 30, 11, 31, 46, 37, 52, 49, 45, 51, 29, 23, 54, 53, 44, 36, 60, 58, 59, 21, 14, 5, 24, 12, 7, 22, 4, 15, 8, 6, 16 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 33, 38, 13, 26, 32, 17, 41, 48, 28, 12, 49, 50, 55, 14, 59, 51, 4, 29, 6, 40, 31, 42, 60, 56, 43, 44, 57, 11, 47, 37, 21, 9, 15, 39, 46, 54, 36, 7, 52, 5, 8, 22, 25, 16, 35, 58, 24, 30, 45, 53, 23 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 42, 11, 21, 22, 33, 50, 12, 32, 28, 14, 6, 35, 29, 24, 9, 23, 15, 8, 38, 17, 31, 7, 30, 4, 39, 45, 46, 26, 16, 53, 58, 36, 43, 57, 44, 37, 56, 49, 51, 59, 47, 5, 52, 55, 54, 40, 25, 41, 60, 13, 48 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 42, 11, 26, 9, 43, 31, 17, 13, 41, 52, 25, 50, 21, 12, 51, 40, 5, 6, 14, 28, 32, 7, 37, 4, 59, 58, 39, 46, 8, 49, 55, 48, 23, 33, 38, 53, 56, 47, 54, 44, 60, 45, 35, 30, 36, 57, 22, 24, 29, 15, 16 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 42, 11, 28, 43, 33, 32, 38, 50, 35, 39, 49, 46, 26, 17, 51, 60, 47, 4, 13, 23, 45, 48, 9, 52, 31, 36, 21, 54, 44, 6, 25, 22, 40, 41, 53, 29, 24, 14, 15, 55, 12, 5, 57, 37, 56, 16, 58, 59, 30, 8, 7 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 42, 11, 40, 9, 17, 50, 12, 5, 21, 33, 43, 36, 32, 28, 52, 44, 23, 41, 26, 31, 37, 35, 22, 47, 29, 13, 54, 49, 4, 16, 24, 14, 39, 30, 48, 60, 7, 6, 59, 8, 58, 46, 25, 38, 55, 57, 51, 56, 45, 53, 15 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 42, 11, 40, 39, 38, 48, 47, 51, 26, 32, 33, 17, 60, 46, 31, 25, 56, 4, 21, 41, 54, 13, 12, 5, 43, 35, 49, 58, 36, 44, 52, 45, 55, 22, 50, 28, 23, 57, 53, 16, 14, 7, 24, 30, 59, 29, 37, 9, 6, 15, 8 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 42, 11, 40, 48, 28, 51, 12, 23, 60, 4, 21, 35, 17, 9, 26, 33, 22, 43, 39, 59, 56, 50, 24, 25, 44, 30, 41, 32, 5, 52, 38, 16, 36, 14, 6, 47, 29, 55, 54, 13, 49, 57, 31, 15, 7, 46, 45, 8, 58, 37, 53 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 42, 50, 12, 41, 43, 21, 33, 38, 59, 44, 52, 28, 29, 60, 32, 30, 11, 26, 17, 9, 24, 40, 35, 39, 37, 4, 13, 48, 49, 58, 25, 46, 55, 51, 47, 23, 36, 53, 5, 16, 22, 14, 54, 57, 6, 45, 56, 31, 15, 8, 7 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 42, 50, 33, 40, 51, 41, 49, 48, 28, 23, 12, 21, 56, 39, 26, 58, 38, 60, 52, 44, 35, 55, 11, 37, 16, 17, 54, 4, 46, 32, 13, 36, 43, 6, 59, 57, 14, 25, 47, 15, 45, 9, 30, 29, 31, 22, 8, 53, 24, 7, 5 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 34, 42, 50, 51, 52, 21, 28, 26, 9, 32, 38, 41, 39, 43, 14, 40, 12, 25, 48, 23, 49, 11, 17, 58, 22, 57, 37, 13, 31, 45, 16, 59, 5, 4, 47, 35, 30, 7, 29, 60, 36, 15, 44, 54, 33, 6, 55, 56, 46, 53, 24, 8 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 41, 17, 9, 21, 14, 42, 33, 13, 22, 15, 25, 31, 6, 26, 16, 5, 12, 39, 34, 23, 28, 29, 46, 38, 35, 8, 45, 30, 4, 43, 52, 53, 51, 32, 47, 60, 36, 11, 24, 55, 48, 40, 7, 56, 50, 58, 57, 59, 49, 44, 37, 54 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 41, 17, 9, 21, 25, 49, 12, 26, 22, 33, 40, 13, 14, 23, 29, 4, 58, 39, 5, 31, 30, 32, 47, 54, 55, 56, 48, 57, 38, 15, 42, 7, 8, 50, 51, 34, 53, 60, 37, 36, 44, 24, 6, 45, 43, 16, 35, 59, 46, 11, 28, 52 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 41, 17, 9, 26, 11, 12, 4, 34, 25, 33, 43, 50, 59, 5, 32, 35, 36, 31, 13, 52, 29, 57, 28, 6, 14, 39, 46, 42, 44, 49, 40, 23, 58, 22, 21, 51, 7, 15, 38, 37, 48, 8, 24, 55, 16, 60, 30, 56, 47, 45, 53, 54 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 41, 17, 9, 26, 11, 21, 32, 38, 14, 42, 43, 49, 57, 4, 12, 28, 37, 33, 46, 45, 34, 40, 22, 52, 47, 35, 13, 29, 6, 7, 15, 50, 31, 25, 58, 54, 53, 5, 36, 8, 48, 59, 23, 16, 30, 51, 55, 56, 39, 60, 24, 44 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 41, 17, 9, 26, 11, 33, 38, 34, 22, 39, 12, 13, 25, 15, 21, 45, 23, 14, 28, 49, 50, 30, 6, 51, 8, 7, 40, 31, 57, 35, 59, 47, 43, 52, 37, 4, 42, 29, 53, 58, 54, 5, 32, 48, 44, 60, 36, 46, 55, 56, 24, 16 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 41, 17, 9, 26, 25, 42, 34, 50, 48, 12, 5, 11, 33, 40, 39, 32, 49, 4, 58, 6, 31, 51, 13, 57, 56, 35, 21, 55, 44, 37, 30, 28, 43, 52, 60, 47, 59, 29, 23, 24, 22, 16, 38, 36, 15, 45, 54, 14, 8, 46, 53, 7 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 41, 17, 9, 42, 25, 49, 50, 58, 59, 28, 43, 34, 22, 48, 57, 40, 39, 12, 11, 51, 13, 35, 52, 47, 36, 44, 29, 38, 46, 4, 37, 21, 33, 26, 14, 53, 30, 31, 23, 56, 55, 5, 32, 45, 6, 54, 7, 15, 16, 24, 60, 8 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 41, 17, 11, 4, 9, 21, 25, 33, 12, 13, 26, 31, 14, 40, 32, 38, 48, 5, 39, 15, 16, 42, 45, 37, 49, 56, 51, 7, 28, 6, 57, 35, 29, 23, 47, 34, 22, 58, 30, 53, 8, 46, 55, 43, 50, 60, 44, 36, 59, 24, 54, 52 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 41, 17, 11, 12, 9, 25, 13, 34, 35, 6, 21, 32, 5, 50, 37, 14, 59, 43, 51, 52, 4, 26, 31, 29, 36, 40, 23, 38, 48, 15, 7, 42, 28, 60, 22, 30, 47, 16, 39, 56, 8, 44, 33, 54, 55, 24, 53, 46, 45, 49, 57, 58 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 41, 17, 11, 12, 9, 40, 13, 33, 39, 21, 26, 48, 22, 34, 50, 6, 25, 49, 5, 47, 32, 37, 56, 51, 29, 14, 42, 28, 59, 55, 15, 58, 35, 16, 43, 30, 24, 31, 54, 52, 57, 46, 36, 23, 45, 38, 8, 4, 60, 53, 44, 7 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 41, 17, 11, 12, 9, 42, 4, 28, 43, 26, 34, 21, 29, 50, 25, 36, 48, 31, 59, 33, 40, 57, 13, 30, 58, 5, 6, 55, 44, 39, 47, 38, 35, 14, 32, 37, 24, 60, 56, 51, 46, 7, 15, 23, 16, 53, 22, 8, 49, 52, 45, 54 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 41, 17, 11, 42, 9, 26, 32, 12, 34, 50, 49, 38, 51, 60, 25, 59, 58, 35, 4, 31, 44, 28, 33, 57, 21, 13, 56, 23, 14, 55, 40, 15, 45, 30, 48, 5, 29, 22, 16, 46, 36, 39, 7, 52, 54, 8, 47, 37, 24, 53, 6, 43 ],
    [ 20, 19, 18, 10, 2, 1, 27, 3, 41, 17, 11, 50, 9, 12, 49, 4, 13, 28, 26, 34, 51, 21, 42, 40, 39, 32, 25, 5, 23, 59, 6, 35, 33, 47, 54, 56, 22, 48, 36, 16, 55, 52, 58, 57, 46, 29, 30, 24, 60, 43, 44, 38, 31, 7, 8, 53, 14, 15, 45, 37 ],
]

In [4]:
board_seqs_int = torch.tensor(data, dtype=torch.long)

num_games, length_of_game = board_seqs_int.shape
print("Number of games:", num_games,)
print("Length of game:", length_of_game)

Number of games: 100
Length of game: 60


In [5]:
stoi_indices = [
    0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 29, 30, 31, 32, 33, 34, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63,
]
alpha = "ABCDEFGH"


def to_board_label(i):
    return f"{alpha[i//8]}{i%8}"


board_labels = list(map(to_board_label, stoi_indices))

In [8]:
board_labels[20]

'C4'